In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'Newtraining.txt'

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 18706 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

he said the way my blue eyes shined put those georgia stars to shame that night i said  "that's a lie" just a boy in a chevy truck that had a tendency of gettin' stuck on back roads at night and i was right there beside him all summer long and then t


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

31 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '"' :   1,
  "'" :   2,
  '-' :   3,
  '4' :   4,
  '5' :   5,
  'a' :   6,
  'b' :   7,
  'c' :   8,
  'd' :   9,
  'e' :  10,
  'f' :  11,
  'g' :  12,
  'h' :  13,
  'i' :  14,
  'j' :  15,
  'k' :  16,
  'l' :  17,
  'm' :  18,
  'n' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'he said the w' ---- characters mapped to int ---- > [13 10  0 23  6 14  9  0 24 13 10  0 27]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

h
e
 
s
a


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'he said the way my blue eyes shined put those georgia stars to shame that night i said  "that\'s a lie'
'" just a boy in a chevy truck that had a tendency of gettin\' stuck on back roads at night and i was r'
'ight there beside him all summer long and then the time we woke up to find that summer gone  but when'
' you think tim mcgraw i hope you think my favorite song the one we danced to all night long the moon '
'like a spotlight on the lake when you think happiness i hope you think that little black dress think '


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'he said the way my blue eyes shined put those georgia stars to shame that night i said  "that\'s a li'
Target data: 'e said the way my blue eyes shined put those georgia stars to shame that night i said  "that\'s a lie'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 13 ('h')
  expected output: 10 ('e')
Step    1
  input: 10 ('e')
  expected output: 0 (' ')
Step    2
  input: 0 (' ')
  expected output: 23 ('s')
Step    3
  input: 23 ('s')
  expected output: 6 ('a')
Step    4
  input: 6 ('a')
  expected output: 14 ('i')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 31) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           7936      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 31)            31775     
Total params: 3,978,015
Trainable params: 3,978,015
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([ 5, 16,  5,  3,  1,  8,  1,  3, 11,  4, 27,  4,  7,  6,  0, 19, 10,
       24,  3,  2,  8, 14, 13, 15, 18,  5, 25, 27,  4, 12,  7, 25,  2, 12,
       26, 29, 16, 17, 14, 16, 29, 15, 22,  9, 27, 10, 17, 22,  3,  4, 29,
       24,  1, 13, 26, 11,  7,  6,  2, 16,  3,  1, 22,  3, 23, 11,  3,  3,
       27, 25, 18, 30,  1, 16,  4,  4,  0,  6, 23, 23,  5, 13, 12,  4, 13,
       13, 17,  0,  4, 30, 22, 14, 27,  8, 29, 16, 28,  0,  7,  0],
      dtype=int64)

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 " my distance but you still catch my eye tell me  baby  do you recognize me well  it's been a year  i"

Next Char Predictions: 
 '5k5-"c"-f4w4ba net-\'cihjm5uw4gbu\'gvyklikyjrdwelr-4yt"hvfba\'k-"r-sf--wumz"k44 ass5hg4hhl 4zriwcykx b '


In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 31)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.434122


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'taylorswift_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS=700

In [27]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 2 steps
Epoch 1/700
2/2 [==============================] - 8s 4s/step - loss: 3.4107
Epoch 2/700
2/2 [==============================] - 6s 3s/step - loss: 3.1912
Epoch 3/700
2/2 [==============================] - 6s 3s/step - loss: 7.8363
Epoch 4/700
2/2 [==============================] - 6s 3s/step - loss: 3.2039
Epoch 5/700
2/2 [==============================] - 6s 3s/step - loss: 3.3021
Epoch 6/700
2/2 [==============================] - 6s 3s/step - loss: 3.3117
Epoch 7/700
2/2 [==============================] - 6s 3s/step - loss: 3.3039
Epoch 8/700
2/2 [==============================] - 7s 4s/step - loss: 3.2851
Epoch 9/700
2/2 [==============================] - 7s 3s/step - loss: 3.2586
Epoch 10/700
2/2 [==============================] - 8s 4s/step - loss: 3.2248
Epoch 11/700
2/2 [==============================] - 8s 4s/step - loss: 3.1827
Epoch 12/700
2/2 [==============================] - 9s 4s/step - loss: 3.1260
Epoch 13/700
2/2 [==============================] - 9s 

2/2 [==============================] - 7s 4s/step - loss: 1.0423
Epoch 119/700
2/2 [==============================] - 7s 4s/step - loss: 1.0081
Epoch 120/700
2/2 [==============================] - 7s 4s/step - loss: 0.9511
Epoch 121/700
2/2 [==============================] - 7s 3s/step - loss: 0.9551
Epoch 122/700
2/2 [==============================] - 7s 4s/step - loss: 0.9092
Epoch 123/700
2/2 [==============================] - 7s 4s/step - loss: 0.9139
Epoch 124/700
2/2 [==============================] - 7s 4s/step - loss: 0.8953
Epoch 125/700
2/2 [==============================] - 7s 3s/step - loss: 0.8573
Epoch 126/700
2/2 [==============================] - 7s 3s/step - loss: 0.8505
Epoch 127/700
2/2 [==============================] - 7s 3s/step - loss: 0.8165
Epoch 128/700
2/2 [==============================] - 7s 4s/step - loss: 0.8097
Epoch 129/700
2/2 [==============================] - 6s 3s/step - loss: 0.7809
Epoch 130/700
2/2 [==============================] - 6s 3s/step - 

2/2 [==============================] - 7s 4s/step - loss: 0.1370
Epoch 222/700
2/2 [==============================] - 7s 4s/step - loss: 0.1354
Epoch 223/700
2/2 [==============================] - 7s 4s/step - loss: 0.1329
Epoch 224/700
2/2 [==============================] - 8s 4s/step - loss: 0.1311
Epoch 225/700
2/2 [==============================] - 7s 4s/step - loss: 0.1316
Epoch 226/700
2/2 [==============================] - 7s 3s/step - loss: 0.1308
Epoch 227/700
2/2 [==============================] - 7s 4s/step - loss: 0.1293
Epoch 228/700
2/2 [==============================] - 7s 4s/step - loss: 0.1319
Epoch 229/700
2/2 [==============================] - 7s 4s/step - loss: 0.1268
Epoch 230/700
2/2 [==============================] - 7s 4s/step - loss: 0.1265
Epoch 231/700
2/2 [==============================] - 8s 4s/step - loss: 0.1370
Epoch 232/700
2/2 [==============================] - 7s 4s/step - loss: 0.1276
Epoch 233/700
2/2 [==============================] - 7s 4s/step - 

2/2 [==============================] - 7s 4s/step - loss: 0.0894
Epoch 325/700
2/2 [==============================] - 7s 4s/step - loss: 0.0856
Epoch 326/700
2/2 [==============================] - 7s 4s/step - loss: 0.0882
Epoch 327/700
2/2 [==============================] - 7s 4s/step - loss: 0.0904
Epoch 328/700
2/2 [==============================] - 7s 4s/step - loss: 0.0897
Epoch 329/700
2/2 [==============================] - 7s 4s/step - loss: 0.0868
Epoch 330/700
2/2 [==============================] - 7s 4s/step - loss: 0.0866
Epoch 331/700
2/2 [==============================] - 7s 4s/step - loss: 0.0847
Epoch 332/700
2/2 [==============================] - 8s 4s/step - loss: 0.0868
Epoch 333/700
2/2 [==============================] - 8s 4s/step - loss: 0.0887
Epoch 334/700
2/2 [==============================] - 7s 4s/step - loss: 0.0819
Epoch 335/700
2/2 [==============================] - 7s 4s/step - loss: 0.0840
Epoch 336/700
2/2 [==============================] - 7s 4s/step - 

2/2 [==============================] - 7s 4s/step - loss: 0.0665
Epoch 438/700
2/2 [==============================] - 7s 3s/step - loss: 0.0617
Epoch 439/700
2/2 [==============================] - 7s 3s/step - loss: 0.0658
Epoch 440/700
2/2 [==============================] - 7s 3s/step - loss: 0.0703
Epoch 441/700
2/2 [==============================] - 7s 3s/step - loss: 0.0648
Epoch 442/700
2/2 [==============================] - 7s 4s/step - loss: 0.0639
Epoch 443/700
2/2 [==============================] - 7s 3s/step - loss: 0.0666
Epoch 444/700
2/2 [==============================] - 7s 4s/step - loss: 0.0656
Epoch 445/700
2/2 [==============================] - 7s 3s/step - loss: 0.0670
Epoch 446/700
2/2 [==============================] - 7s 3s/step - loss: 0.0655
Epoch 447/700
2/2 [==============================] - 7s 3s/step - loss: 0.0674
Epoch 448/700
2/2 [==============================] - 7s 3s/step - loss: 0.0671
Epoch 449/700
2/2 [==============================] - 7s 4s/step - 

2/2 [==============================] - 7s 3s/step - loss: 0.0632
Epoch 540/700
2/2 [==============================] - 7s 3s/step - loss: 0.0610
Epoch 541/700
2/2 [==============================] - 7s 3s/step - loss: 0.0614
Epoch 542/700
2/2 [==============================] - 7s 4s/step - loss: 0.0579
Epoch 543/700
2/2 [==============================] - 7s 3s/step - loss: 0.0572
Epoch 544/700
2/2 [==============================] - 7s 3s/step - loss: 0.0582
Epoch 545/700
2/2 [==============================] - 7s 3s/step - loss: 0.0572
Epoch 546/700
2/2 [==============================] - 7s 3s/step - loss: 0.0604
Epoch 547/700
2/2 [==============================] - 7s 3s/step - loss: 0.0588
Epoch 548/700
2/2 [==============================] - 7s 3s/step - loss: 0.0584
Epoch 549/700
2/2 [==============================] - 7s 4s/step - loss: 0.0562
Epoch 550/700
2/2 [==============================] - 7s 3s/step - loss: 0.0552
Epoch 551/700
2/2 [==============================] - 7s 3s/step - 

2/2 [==============================] - 7s 4s/step - loss: 0.0516
Epoch 643/700
2/2 [==============================] - 7s 4s/step - loss: 0.0526
Epoch 644/700
2/2 [==============================] - 7s 3s/step - loss: 0.0540
Epoch 645/700
2/2 [==============================] - 7s 4s/step - loss: 0.0537
Epoch 646/700
2/2 [==============================] - 7s 4s/step - loss: 0.0531
Epoch 647/700
2/2 [==============================] - 7s 4s/step - loss: 0.0521
Epoch 648/700
2/2 [==============================] - 7s 3s/step - loss: 0.0511
Epoch 649/700
2/2 [==============================] - 7s 3s/step - loss: 0.0530
Epoch 650/700
2/2 [==============================] - 7s 4s/step - loss: 0.0514
Epoch 651/700
2/2 [==============================] - 7s 4s/step - loss: 0.0499
Epoch 652/700
2/2 [==============================] - 9s 4s/step - loss: 0.0454
Epoch 653/700
2/2 [==============================] - 8s 4s/step - loss: 0.0523
Epoch 654/700
2/2 [==============================] - 8s 4s/step - 

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'taylorswift_training_checkpoints\\ckpt_700'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            7936      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 31)             31775     
Total params: 3,978,015
Trainable params: 3,978,015
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"something in the way"))

something in the way you lad when i didn't wonder how you are tonight cauce there in the back of your mind i shouldn't be asking myself  "why" you shouldn't be begging for forgiveness at my feet you should've said "no"  baby  and you might still have me  you can see that i've been crying and baby  dear  santa baby so hurry down the chimney tonight  come and trim my christmas tree with some you think tim mcgraw i hope you think my favorite song sor sneakin' out late  tapping on your window when werreason for the teardrops on my guitar the only thing that keeps me wishing on a wishing star he's the song in the car i keep singing don't know why i do  drew walks by me can he tell that i can't breake a smile so he won't see that i want and i'm needing everything that we should be  i'll bet she's litt me i gould go a perfect day just walk away  no use defending words that you will ngvit all year  santa baby so hurry down the chimney tonight caume there in the back of your mind i shouldn't be

In [34]:
import tkinter as tk
import requests

HEIGHT = 500
WIDTH = 600

def test_function(entry):
	print("This is the entry:", entry)


def get_song(city):

	label.insert(tk.INSERT,generate_text(model, start_string=city))



root = tk.Tk()

root.title("Magenta")

canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()

background_image = tk.PhotoImage(file='landscape3.png')
background_label = tk.Label(root, image=background_image)
background_label.place(relwidth=1, relheight=1)

frame = tk.Frame(root, bg='#80c1ff', bd=5)
frame.place(relx=0.5, rely=0.1, relwidth=0.75, relheight=0.1, anchor='n')

entry = tk.Entry(frame, font=("Helvetica", 15))
entry.place(relwidth=0.65, relheight=1)

button = tk.Button(frame, text="Generate Song", font=40, command=lambda: get_song(entry.get()))
button.place(relx=0.7, relheight=1, relwidth=0.3)

lower_frame = tk.Frame(root, bg='#80c1ff', bd=10)
lower_frame.place(relx=0.5, rely=0.25, relwidth=0.75, relheight=0.6, anchor='n')

label = tk.Text(lower_frame,font=("Helvetica", 15))
label.place(relwidth=1, relheight=1)

root.mainloop()